In [1]:
import binascii, gzip, struct, sys, os, numpy as np
from pyspark import SparkConf, SparkContext

In [2]:
trnim = open('train_images.txt', mode='rb').read()
trnlb = open('train_labels.txt', mode='rb').read()
tstim = open('test_images.txt', mode='rb').read()
tstlb = open('test_labels.txt', mode='rb').read()

In [3]:
print(len(trnim), len(trnlb), len(tstim), len(tstlb))

47040016 60008 7840016 10008


In [4]:
trnx = np.zeros(len(trnim)-16)
trny = np.zeros(len(trnlb)-8)
tstx = np.zeros(len(tstim)-16)
tsty = np.zeros(len(tstlb)-8)

In [5]:
n = 60000 # n for inputs

In [6]:
for i in range(16, len(trnim)): trnx[i-16] = str(trnim[i])
for i in range(8, len(trnlb)): trny[i-8] = str(trnlb[i])
for i in range(16, len(tstim)): tstx[i-16] = str(tstim[i])
for i in range(8, len(tstlb)): tsty[i-8] = str(tstlb[i])

In [7]:
trnx = np.reshape(trnx/255, (60000, 784))[0:n]
trny = np.reshape(trny, (60000, 1))[0:n]
tstx = np.reshape(tstx/255, (10000, 784))
tsty = np.reshape(tsty, (10000, 1))

In [8]:
trnt = np.append(np.zeros((n,10)), trnx, axis=1); data = []
for j in range(10): 
    trnt[:,j:j+1] = (trny == j) + 0
    data.append(np.append(trnt[trnt[:,j] == 1], trnt[trnt[:,j] != 1][np.random.randint(0, trnt[trnt[:,j] != 1].shape[0], trnt[trnt[:,j] == 1].shape[0])], axis=0))

In [17]:
memory = '12g'
pyspark_submit_args = ' --driver-memory ' + memory + ' pyspark-shell'
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args
conf = SparkConf()
sc = SparkContext(conf=conf)

points = sc.parallelize(trnt)
#points = []
#for j in range(10): points.append(sc.parallelize(data[j])) 
iterations = 50; D = 784; C = 10 # C for classes, D for dimensions

# Initialize w to a random value
w = (np.random.ranf((C,D)) - 0.5)/10

# Compute logistic regression gradient for a matrix of data points
def gradient(X, Y, w, j):
    X = np.reshape(X, (1, D))
    return np.ravel((1.0 / (1.0 + np.exp(Y * X.dot(w[j])))) * Y * X.T)
    #return np.ravel((X * ((1.0 / (1.0 + np.exp(X.dot(w[j])))) - Y)))
    #return np.ravel((1.0 / (1.0 + np.exp(Y - X.dot(w[j])))) * Y * X.T)
    #return np.ravel(X.T * np.exp(X.dot(w[j])) - Y * X.T)
    
def add(x, y):
    x += y
    return x

for i in range(iterations):
    #prnt1 = str(i+1)
    #prnt2 = str(i+1)
    for j in range(C):
        #exx = np.divide(np.exp(trnx.dot(w[j])),1+np.exp(trnx.dot(w[j])))
        #acc = np.sum(trnt[:,j] == (exx >= 0.5))/trnt.shape[0]
        #err = np.sum(np.abs(trnt[:,j] - exx))/trnt.shape[0]
        prnt1 += " | %.4f" % err
        prnt2 += " | %.4f" % acc
        w[j] -= points.map(lambda m: gradient(m[C:], m[j], w, j)).reduce(add)/(n*10)
    #print(prnt1)
    #print(prnt2)
    
exx = np.zeros((n, C))      
for j in range(C): exx[:,j] = np.divide(np.exp(trnx.dot(w[j])),1+np.exp(trnx.dot(w[j])))
acc = np.sum(trny == np.array([np.argmin(exx, axis=1)]).T)/n
print("Training Accuracy: %6f" % acc)

exx = np.zeros((10000, C))
for j in range(C): exx[:,j] = np.divide(np.exp(tstx.dot(w[j])),1+np.exp(tstx.dot(w[j])))
acc = np.sum(tsty == np.array([np.argmin(exx, axis=1)]).T)/tsty.shape[0]
print("Testing Accuracy: %6f" % acc)

sc.stop()


Training Accuracy: 0.668950
Testing Accuracy: 0.675600


In [10]:
(exx[0:1])

array([[2.05039202e-12, 7.73912855e-07, 8.01804065e-11, 9.08832703e-12,
        3.88362163e-11, 5.73609621e-09, 2.06480276e-09, 1.83361142e-17,
        4.42894869e-12, 2.56723177e-13]])

In [11]:
tsty[0:20].T

array([[7., 2., 1., 0., 4., 1., 4., 9., 5., 9., 0., 6., 9., 0., 1., 5.,
        9., 7., 3., 4.]])

In [16]:
sc.stop()

In [15]:
exx = np.zeros((n, C))      
for j in range(C): exx[:,j] = np.divide(np.exp(trnx.dot(w[j])),1+np.exp(trnx.dot(w[j])))
acc = np.sum(trny == np.array([np.argmin(exx, axis=1)]).T)/n
print("Training Accuracy: %6f" % acc)

Training Accuracy: 0.654967


In [16]:
exx = np.zeros((10000, C))
for j in range(C): exx[:,j] = np.divide(np.exp(tstx.dot(w[j])),1+np.exp(tstx.dot(w[j])))
acc = np.sum(tsty == np.array([np.argmin(exx, axis=1)]).T)/tsty.shape[0]
print("Testing Accuracy: %6f" % acc)

Testing Accuracy: 0.666200


In [22]:
exx[0]

array([2.66182513e-03, 6.10921807e-04, 5.98891259e-02, 1.41419325e-07,
       8.29737359e-05, 5.42040857e-06, 5.53224143e-01, 9.94254330e-05,
       1.03674874e-05, 8.06145248e-01])

In [23]:
trny[0]

array([5.])

In [18]:
exx

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])